In [ ]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
stage_claims_dummy = dataiku.Dataset("stage_claims_dummy")
stage_claims_dummy_df = stage_claims_dummy.get_dataframe()

In [ ]:
# Compute recipe outputs from inputs
import random
from faker import Factory
import datetime

faker = Factory.create()

# get configs
create_or_append = dataiku.get_custom_variables()["create_or_append"]
total_claims = int(dataiku.get_custom_variables()["total_claims"])
append_claims = int(dataiku.get_custom_variables()["append_claims"])

#create static vars
claim_current_status = '01, 91, 02, , 01, 02, 01, 02'.split(', ')
provider_specialty = '1234, 1199, 2239, 6555, 1310'.split(',')
procedure_code = '9979, C4391, X123, D4578, 9N278, NSZKA, NYH4O, 43IXK, 3P29S, S3I46, 3K2US, 3272D, S3DSHJ, S2GJ3, 32WSS  ,    SN32K, VCGHD, , WSQQS, NULL,   23SDS, XSMJH   , 23KSWA'.split(', ')
diagnosis_code = 'SJ3YH, SG2IS, SO22S, SI2U3, G654R, LIG5UG, 796IO, 4JKY5, ABVCH, 65P3E, ,DIU3Y   , 43KJJ, 3SDB2   , 231YQ,   21SHJ, HSGZ21, 0, NULL, 2SAGH, S1MHQV, PJO7Y, 654GB, FD3R2, XENW2K,   43EDDS'.split(', ')
provider_id = '   WEFFER,    FKJ4BH3, FHU34KF   , NULL, F3K4HY   , NFKEWJAB, 23EDEW, 6476YHFG, KJH6K5J7Y    , 3KW45UTH, FKJSEBN, 4354FR, 343453VBDF, GH45K3, JGJKJBAT, KGTNRL, 754HTF, HFU8O34W , FE4U8O, LIU34H, '.split(', ')
provider_entity = 'P, P, P, P, P, P, P, I, G, F'.split(', ')

if(create_or_append == "create"):
    num_claims = total_claims
else:
    num_claims = append_claims

def date_between(d1, d2):
    f = '%b%d-%Y'
    return faker.date_time_between_dates(datetime.datetime.strptime(d1, f), datetime.datetime.strptime(d2, f))

# replaces data value with anomaly with defined probability, else return the original data
# Jamie refactor
def add_anomalies(value):
    # make 5 % chance of anomaly
    if random.randint(0, 100) <= 5:
        return value * random.randint(2,6)
    else:
        return value * random.randint(2,6)

def fakerecord_claims(num_claims):

    ## Define variables to generate 2 separate datasets for claims and claim line items
    claimrepeats = [faker.numerify('############') for _ in range(num_claims)]
    lineitemnumbers = [faker.random_int(min = 1, max = 20) for _ in range(num_claims)]
    claims = pd.DataFrame(np.repeat(claimrepeats, lineitemnumbers), columns = ['clcl_id'])
    claims['claim_lineitem_number'] = claims.groupby('clcl_id').cumcount()+1
    cmc_clcl_claim_ls = []
    cmc_cdml_cl_line_ls = []

    ## Define cmc_clcl_claims data (claim level)
    unique_claims = pd.DataFrame(claims.clcl_id.unique(), columns = ['clcl_id'])
    for index, row in unique_claims.iterrows():
        claims_dict = {
            'clcl_id': unique_claims['clcl_id'][index],
            'claim_current_status': random.choice(claim_current_status),
            'provider_id': random.choice(provider_id),
            'clcl_input_dt': date_between('dec31-2016', 'dec31-2017'),
            'new_member_key': faker.random_int(min = 1, max = 67980)

        }
        cmc_clcl_claim_ls.append(claims_dict)

    ## Define cmc_cdml_cl_line data (claim line item level)
    ''' for index, row in claims.iterrows():
        claims_li_dict = {
            'clcl_id': claims['clcl_id'][index],
            'claim_lineitem_number': claims['claim_lineitem_number'][index],
            'procedure_code': random.choice(procedure_code),
            'diagnosis_code': random.choice(diagnosis_code),
            'lineitem_units_charged': add_anomalies(faker.random_int(min = 1, max = 40)),
            'lineitem_units_allowed': add_anomalies(faker.random_int(min = 1, max = 35)),
            'line_item_amount_charged': add_anomalies(faker.pyfloat(left_digits=4, right_digits=2, positive=True)),
            'line_item_paid_amount': add_anomalies(faker.pyfloat(left_digits=4, right_digits=2, positive=True))
        }
        cmc_cdml_cl_line_ls.append(claims_li_dict)
        '''

    return cmc_clcl_claim_ls



stage_claims_hdfsparquet_df = pd.DataFrame(fakerecord_claims(num_claims)) 
# Write recipe outputs
stage_claims_hdfsparquet = dataiku.Dataset("stage_claims_hdfsparquet")
stage_claims_hdfsparquet.write_with_schema(stage_claims_hdfsparquet_df)